# Focault test data reduction

This document will describe step by step the process of extracting the surface profile of a telescope mirror from Foucault or wire test data.

## Derivation of the mirror curve from test data

![title](figures/diagram_1.svg)

$G(x) = \langle x, g(x) \rangle$

$F(x) = \langle 0, f(x) \rangle$

$T(x) = \frac{G'(x)}{|G'(x)|} = \langle\frac{1}{|G'(x)|}, \frac{z'(x)}{|G'(x)|}\rangle$

$F(x) - G(x) = \langle 0 - x, f(x)-g(x) \rangle = \langle -x, f(x) - g(x) \rangle$

$T(x) \cdot (F(x) - G(x)) = 0$

$\left( - \frac{x}{|G'(x)|} + (f(x) - g(x))\frac{g'(x)}{|G'(x)|} \right) = 0$

$(f(x) - g(x))g'(x) = x$

$g'(x) = \frac{x}{f(x) - g(x)}$

### kjhkhkhkjh